### Step 4: Live Recommendation Input

This step allows the user to input any shopping basket manually.
- You can type items separated by commas (`,`).
- The system will output **recommended items** based on the rules and fallback logic.
- Works for baskets that exist in the dataset as well as completely new baskets.


In [3]:
import pickle
import pandas as pd
import ast

# load preprocessed basket list
basket_file = r"C:\coding5final\coding5\data\processed\basket_list.pkl"
with open(basket_file, "rb") as f:
    basket_list = pickle.load(f)

print(f"Number of transactions: {len(basket_list)}")
print("Example basket:", basket_list[0])

# load previously saved association rules
rules_file = r"C:\coding5final\coding5\data\processed\association_rules.csv"
rules = pd.read_csv(rules_file)

# convert antecedents and consequents from string to set
rules['antecedents'] = rules['antecedents'].apply(ast.literal_eval)
rules['consequents'] = rules['consequents'].apply(ast.literal_eval)

print(f"Number of rules loaded: {len(rules)}")


Number of transactions: 14964
Example basket: ['sausage', 'whole milk', 'semi-finished bread', 'yogurt']
Number of rules loaded: 57


In [7]:
def recommend_items_full(basket, rules_df, basket_list, top_n=5):
    basket = set(basket)
    recommendations = {}
    for _, row in rules_df.iterrows():
        if row['antecedents'].issubset(basket):
            for item in row['consequents']:
                if item not in basket:
                    recommendations[item] = max(recommendations.get(item, 0), row['confidence'])
    if not recommendations:
        for _, row in rules_df.iterrows():
            overlap = len(basket & row['antecedents'])
            if overlap > 0:
                for item in row['consequents']:
                    if item not in basket:
                        score = overlap * row['confidence']
                        recommendations[item] = max(recommendations.get(item, 0), score)
    if not recommendations:
        co_occur = {}
        for trans in basket_list:
            trans_set = set(trans)
            if basket & trans_set:
                for item in trans_set:
                    if item not in basket:
                        co_occur[item] = co_occur.get(item, 0) + 1
        sorted_items = sorted(co_occur.items(), key=lambda x: x[1], reverse=True)
        recommendations = {item: count for item, count in sorted_items[:top_n]}
    
    if not recommendations:
        return ["No recommendation found"]
    
    recommended_sorted = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    return [item for item, score in recommended_sorted][:top_n]


In [10]:
# run live recommendation
live_basket_recommendation(rules, basket_list, top_n=5)


Enter basket items separated by commas:  whole milk, yoghurt



Your Basket: {'whole milk', 'yoghurt'}
Recommended Items: ['other vegetables', 'rolls/buns', 'soda', 'yogurt', 'sausage']


## How the Recommendation System Handles Rare Baskets

The recommendation system works in three steps to make sure it can always suggest items, even if the basket is unusual or new.

### 1. Exact Matches
First, the system looks for rules that exactly match the items in the basket.  
For example, if a basket has `pasta` and `olive oil` and there’s a rule that says people who buy these also buy `canned tomatoes`, that recommendation is used.  
This is the most precise way to suggest items because it’s based on real patterns in the data.

### 2. Partial Matches
If no exact rules match, the system looks for partial matches.  
It checks which rules have some items in common with the basket, and gives a score based on how many items overlap and how strong the rule is.  
This way, even if your basket isn’t exactly like the patterns in the dataset, the system can still make reasonable guesses.

### 3. Popular Items Fallback
Finally, if neither exact nor partial matches produce anything, the system falls back on frequency.  
It looks at the dataset to see which items often appear with anything in the basket, and suggests the most common ones.  
This ensures you always get recommendations, even for very unusual combinations.

---

This three-step approach makes the system flexible and reliable, so it can handle both common and rare shopping baskets.
